# Basic RAG Pipeline Implementation


### Intro

This notebook implements a Retrieval-Augmented Generation (RAG) pipeline using classes from src directory. The implementation is divided into individual pipeline stages:


1. Document Processing: Token-aware chunking with overlap
2. Embedding Generation: Semantic vectorization
3. Vector Storage: PostgreSQL + pgvector indexing
4. Retrieval: Cosine similarit*y search
5. Generation: Context-augmented LLM completion

### Components

**TextProcessor** - Text segmentation and token budget management.

* Token-based chunking (512 tokens, 50 token overlap)
* Uses cl100k_base tokenizer for GPT compatibility
* Adaptive context assembly within token budgets

**HuggingFaceClient** - Embedding generation and LLM inference.

* Embeddings: Local sentence-transformers (all-MiniLM-L6-v2, 384-dim)
* Generation: Remote Hugging Face Inference API (default: Mistral-7B-Instruct)

**PgVectorDB** - PostgreSQL interface with vector similarity search.

* Stores embeddings as VECTOR(384) with chunk metadata
* Uses ivfflat indexing for approximate nearest neighbor search
* Cosine similarity search via <=> operator


In [2]:
import logging
import os
import sys
from typing import List, Dict, Optional
# from pathlib import Path


# import psycopg2
# from psycopg2.extras import execute_values, Json
# from pgvector.psycopg2 import register_vector
# from huggingface_hub import InferenceClient
# from transformers import pipeline


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


sys.path.insert(0, '../src/')


from dotenv import load_dotenv

# Setup
load_dotenv()


True

In [3]:
from text_processor import TextProcessor
from pgvector_client import PgVectorClient
from hf_client import HuggingFaceClient
# from rag_handler import PgVectorRAG


In [5]:
PG_CONN_STRING = os.getenv("PG_CONNECTION_STRING")
HF_TOKEN= os.getenv("HF_TOKEN")

file_paths = [
    # "../documents/policy.txt",
    # "../documents/basic_info.md",
    # Add more files
    # "../documents/cover_letter.txt",
    # "../documents/cv_info_basic.txt",
    # "../documents/cv_info_extensive.txt",
    # "../documents/cv_info_extensive_first_person.txt"

    "../documents/contact_info_chunk_750.txt",
    "../documents/hr_qa_chunk_750.txt",
    "../documents/projects_chunk_750.txt",
    "../documents/motivation_chunk_750.txt",
]

# BATCH_SIZE=32 # batch_size=
# EMBEDDING_DIM=384 # embedding_dim

# CHUNK_SIZE=700
# CHUNK_OVERLAP=150
# MAX_CONTEXT_TOKENS=3500
# TEMPERATURE=0.2

# Chunking
CHUNK_SIZE = 750  # characters (not tokens)
CHUNK_OVERLAP = 125  # ~17% overlap
# MIN_CHUNK_SIZE = 200  # Prevent tiny chunks

# Retrieval
TOP_K = 6  # Retrieve top 6 most similar chunks
SIMILARITY_THRESHOLD = 0.50  # Cosine similarity cutoff
MAX_CONTEXT_TOKENS = 3500  # For Mistral-7B's 8k context window

# Generation
TEMPERATURE = 0.2  # Low temperature for factual accuracy
MAX_NEW_TOKENS = 500  # Sufficient for detailed CV answers
# TOP_P = 0.9  # Nucleus sampling
# REPETITION_PENALTY = 1.1  # Prevent redundancy


# EMBEDDING_MODEL="sentence-transformers/all-MiniLM-L6-v2"
LLM_MODEL="mistralai/Mistral-7B-Instruct-v0.2"# "mistralai/Mistral-7B-Instruct-v0.2"


EMBEDDING_MODEL = "BAAI/bge-small-en-v1.5"  # ~30% better retrieval for factual Q&A
EMBEDDING_DIM = 384  # Keep same dimension
# LLM_MODEL = "mistralai/Mistral-7B-Instruct-v0.3" 

#  Query the system
questions = [
    "What is mario's email?",
    "How long does shipping take?",
    "Where there any projects with recommendation systems done by Mario?",
    "Does mario like data science?"
]


SIMILARITY_THRESHOLD=0.2
K=10

In [6]:
# Initialize components 
text_processor = TextProcessor(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    max_context_tokens=MAX_CONTEXT_TOKENS
)

db_client = PgVectorClient(
    connection_string=PG_CONN_STRING,
    embedding_dim=EMBEDDING_DIM  # Match embedding model output
)

hf_client = HuggingFaceClient(
    hf_token=HF_TOKEN,
    embedding_model=EMBEDDING_MODEL,
    llm_model=LLM_MODEL,
    # use_remote_llm=True
)



INFO:pgvector_client:Database connected
INFO:pgvector_client:pgvector extension enabled
INFO:pgvector_client:Database schema created
INFO:hf_client:Initialized embedding model: BAAI/bge-small-en-v1.5
INFO:hf_client:Initialized LLM: mistralai/Mistral-7B-Instruct-v0.2


## Load data

In [7]:
all_chunks = [] # list of dicts. Each dict is a 

# Process each file
for file_path in file_paths:
    chunks = text_processor.chunk_file(file_path)
    all_chunks.extend(chunks)
    logger.info(f"Processed {file_path}: {len(chunks)} chunks")
    
    logger.info(f"chunk length {len(chunks)}, {chunks[:10]}")
    logger.info(f"total chunks: {len(all_chunks)}, {all_chunks[-10:]}")

logger.info(f"Total chunks from all files: {len(all_chunks)}")


INFO:text_processor:Chunked contact_info_chunk_750.txt: 3 chunks
INFO:__main__:Processed ../documents/contact_info_chunk_750.txt: 3 chunks
INFO:__main__:chunk length 3, [{'content': "================================================================================\nCHUNK_01_COMPLETE_CONTACT_INFORMATION\n================================================================================\nMargot Razumeyeva (also known as Margo Razumeyeva) - Professional Contact Details\n\nI'm Margot Razumeyeva, a Machine Learning Engineer based in Amsterdam, Netherlands. The best way to reach me is by email at margo.razumeyeva@gmail.com - that's my preferred contact method for all professional correspondence, recruitment opportunities, interview scheduling, and business communication. My phone number is available upon request and will be provided to qualified employers during the interview process.\n\nI'm currently located in Amsterdam, Netherlands, and I'm available for work opportunities in Amsterdam and s

In [8]:
all_chunks [0]

{'content': "================================================================================\nCHUNK_01_COMPLETE_CONTACT_INFORMATION\n================================================================================\nMargot Razumeyeva (also known as Margo Razumeyeva) - Professional Contact Details\n\nI'm Margot Razumeyeva, a Machine Learning Engineer based in Amsterdam, Netherlands. The best way to reach me is by email at margo.razumeyeva@gmail.com - that's my preferred contact method for all professional correspondence, recruitment opportunities, interview scheduling, and business communication. My phone number is available upon request and will be provided to qualified employers during the interview process.\n\nI'm currently located in Amsterdam, Netherlands, and I'm available for work opportunities in Amsterdam and surrounding areas. I'm open to on-site, hybrid, and remote positions within the Netherlands market.\n\nCommon Questions:\nQ: What is Margot's email? A: margo.razumeyeva@gm

In [9]:
# Generate embeddings for all chunks
texts = [chunk['content'] for chunk in all_chunks]
embeddings = hf_client.get_embeddings(texts)

logger.info(f"Generated embeddings for {len(embeddings)} chunks")

# Add embeddings to chunks
for i, chunk in enumerate(all_chunks):
    embedding = embeddings[i]
    if hasattr(embedding, 'tolist'):
        embedding = embedding.tolist()
    chunk['embedding'] = embedding
    # logger.info(f"Generated embeddings for {len(embeddings)} chunks")



# #  Generate embeddings in batches (HuggingFaceClient)
# texts = [chunk["content"] for chunk in chunks]
# embeddings = []

# for i in range(0, len(texts), batch_size):
#     batch = texts[i:i + batch_size]
#     batch_embeddings = hf_client.get_embeddings(batch)
#     embeddings.extend(batch_embeddings)   # adds each embedding vector
#     logger.info(f"Embedded batch {i // batch_size + 1}/{(len(texts) + batch_size - 1) // batch_size}")

INFO:hf_client:Generated 24 embeddings
INFO:__main__:Generated embeddings for 24 chunks


In [10]:

# Insert all chunks
db_client.insert_chunks(all_chunks)
logger.info(f"Inserted {len(all_chunks)} chunks into database")


INFO:pgvector_client:Inserted 24 chunks
INFO:__main__:Inserted 24 chunks into database


In [11]:
for ind, chunk in enumerate(chunks):
    logger.info(f"chunk number {ind}, chunk len {len(chunk['content'])}: {chunk}")
    logger.info("\n")

INFO:__main__:chunk number 0, chunk len 4647: {'content': "================================================================================\nCHUNK_01_MARGOT_CORE_IDENTITY_AND_MOTIVATION\n================================================================================\nWho Is Margot and What Drives Her?\n\n\nWhat really motivates me is understanding how systems work at their deepest level. I'm energized by the challenge of taking complex, interconnected components and uncovering how they interact, where inefficiencies exist, and how they can be optimized to deliver greater value. For me, the most fulfilling aspect of my work isn't simply implementing solutions - it's truly comprehending the entire ecosystem in which they operate. I find genuine satisfaction in diving into engineering challenges, understanding data flows, system architectures, business processes, and user needs to build solutions that are not just functional, but elegant and efficient.\n\nThis curiosity extends beyond th

## Query the model - one question

### Create embeddings

In [19]:
llm_question="What is margot's email?" #"refund policy"

print( hf_client.get_embeddings([llm_question]))

query_embedding = hf_client.get_embeddings([llm_question])[0]
# if not isinstance(query_embedding, list):
#     query_embedding = query_embedding.tolist()
#query_embedding

chunks = db_client.search(query_embedding, 
    k=K, 
    similarity_threshold=0.4)
    #=SIMILARITY_THRESHOLD)
logger.info(f"Found: {len(chunks)} results")

# logger.info(chunks)

# llm_question="what is this cover letter about?"
context=chunks[0]['content']
logger.info(f"context: {context}")

hf_client.generate_answer(
        question=llm_question,
        context=context,
        max_new_tokens=MAX_CONTEXT_TOKENS,
        temperature=TEMPERATURE)


INFO:hf_client:Generated 1 embeddings


[[-0.037040095776319504, 0.02660325914621353, -0.028648292645812035, -0.018384834751486778, -0.011772776022553444, -0.06498906016349792, 0.04760832339525223, 0.06902263313531876, 0.03305904194712639, 0.01568908616900444, 0.05042333900928497, -0.04832042008638382, 0.012172654271125793, 0.013113553635776043, 0.026877984404563904, 0.03971336781978607, 0.06788039207458496, -0.05963828042149544, -0.03172103688120842, 0.03486816957592964, -0.04030569642782211, -0.06106934696435928, -0.034911926835775375, 0.03390366956591606, -0.003661578521132469, 0.0203623678535223, -0.030595986172556877, 0.024825669825077057, -0.13616211712360382, -0.09331941604614258, 0.027046017348766327, 0.008679287508130074, 0.019176239147782326, 0.045655447989702225, 0.015225772745907307, 0.0037607657723128796, 0.011677793227136135, -0.002297110389918089, -0.051521915942430496, -0.023656534031033516, 0.0004690517671406269, -0.06434041261672974, -0.01472102478146553, 0.0002387745480518788, 0.02967112511396408, -0.09653

INFO:hf_client:Generated 1 embeddings
INFO:__main__:Found: 10 results
INFO:__main__:context: ================================================================================
CHUNK_01_COMPLETE_CONTACT_INFORMATION
Margot Razumeyeva (also known as Margo Razumeyeva) - Professional Contact Details

I'm Margot Razumeyeva, a Machine Learning Engineer based in Amsterdam, Netherlands. The best way to reach me is by email at margo.razumeyeva@gmail.com - that's my preferred contact method for all professional correspondence, recruitment opportunities, interview scheduling, and business communication. My phone number is available upon request and will be provided to qualified employers during the interview process.

I'm currently located in Amsterdam, Netherlands, and I'm available for work opportunities in Amsterdam and surrounding areas. I'm open to on-site, hybrid, and remote positions within the Netherlands market.

Common Questions:
Q: What is Margot's email? A: margo.razumeyeva@gmail.com
Q: 

'The answer to the question is: margo.razumeyeva@gmail.com\n\nMargot has explicitly stated that this is her preferred email address for professional correspondence, including job opportunities, interview scheduling, and business communication.'

In [6]:
# llm_question="what is this cover letter about?"
llm_question="Who is margot?"

query_embedding = hf_client.get_embeddings([llm_question])[0]
# if not isinstance(query_embedding, list):
#     query_embedding = query_embedding.tolist()
#query_embedding

chunks = db_client.search(query_embedding, 
    k=K, 
    similarity_threshold=0.4)
    #=SIMILARITY_THRESHOLD)
logger.info(f"Found: {len(chunks)} results")

context = text_processor.assemble_context(chunks, question=llm_question)
logger.info(context)
# context=chunks[0]['content']

hf_client.generate_answer(
        question=llm_question,
        context=context,
        max_new_tokens=MAX_CONTEXT_TOKENS,
        temperature=0.7)

INFO:hf_client:Generated 1 embeddings
INFO:__main__:Found: 1 results
INFO:text_processor:Context: 116/1345 tokens, 1/1 chunks
INFO:__main__:[cover_letter.txt | Chunk 5 | Similarity 0.433]
. She is energized by challenges that require deep understanding, efficient system design, and effective teamwork to solve.

Looking forward, she seeks opportunities to continue building sophisticated machine learning systems while working with teams who value technical excellence, business focus, and collaborative culture. She is excited by the evolving landscape of machine learning and the opportunity to apply emerging technologies to create increasingly valuable solutions.

Above all, Margot is driven by the satisfaction of understanding complex systems, building solutions that work elegantly and efficiently, and collaborating with people to deliver outcomes that matter.

END_OF_DOCUMENT

INFO:hf_client:Generated answer (551 chars)


'Margot is a person who is energized by complex challenges and enjoys building sophisticated machine learning systems. She values working in teams that prioritize technical excellence, business focus, and a collaborative culture. Margot is excited about the rapidly evolving landscape of machine learning and the opportunity to apply new technologies to create valuable solutions. Above all, she finds satisfaction in understanding complex systems, creating elegant and efficient solutions, and collaborating with others to deliver meaningful outcomes.'

In [12]:
llm_question="what is this cover letter about?"
context=chunks[0]['content']
print(context)

hf_client.generate_answer(
        question=llm_question,
        context=context,
        max_new_tokens=MAX_CONTEXT_TOKENS,
        temperature=0.7)

# Cover letter

## Basic info


This is a document with CV summary


Name: Mario (Marvin) Theplumber

email: mario.thplumber@gmail.com
phone: +72787226083

North Holland, Netherlands
https://github.com/razmarrus
https://www.linkedin.com/in/razmarrus/

npm i jsonresume-theme-caffeine-tweaked
resume export --theme caffeine-tweaked resume.pdf

adress: Beethovenstraat 22-1 1099 LK Rotterdam
Netherlands 

Education: Brazil State University of Informatics


## About me / Role description

As a results-oriented Data Scientist, I believe that combining Machine Learning and Statistics provides businesses with solid answers to their questions. While staying close to the data, I work closely with my business-oriented colleagues to translate business goals into achievable objectives using models.

My focus is on customer and financial analytics, where I assist in improving marketing strategies with a better understanding of customers’ preferences and measuring the effectiveness of promotion with s

INFO:hf_client:Generated answer (620 chars)


'This cover letter is about Mario (Marvin) Theplumber introducing himself as a results-oriented Data Scientist with a focus on customer and financial analytics. He mentions his expertise in the full pipeline delivery of data science projects, specifically in the area of creating recommendation systems. The letter describes a project where he developed a multi-channel recommendation system for a pharmaceutical client using Machine Learning models and staying close to the business goals. He also mentions his love for the data science process and his current residence in Amsterdam with a Netherlands residence permit.'

In [ ]:
# context_text = "\n".join([d["content"][:400] for d in chunks])
# prompt = f"Context: {context_text[:600]}\n\nQ: what is in {question}?\nA:"
# prompt

'Context: # Cover letter\n\n## Basic info\n\n\nThis is a document with CV summary\n\n\nName: Mario (Marvin) Theplumber\n\nemail: mario.thplumber@gmail.com\nphone: +72787226083\n\nNorth Holland, Netherlands\nhttps://github.com/razmarrus\nhttps://www.linkedin.com/in/razmarrus/\n\nnpm i jsonresume-theme-caffeine-tweaked\nresume export --theme caffeine-tweaked resume.pdf\n\nadress: Beethovenstraat 22-1 1099 LK Rotterdam\nNetherland\n# Cover letter\n\n## Basic info\n\n\nThis is a document with CV summary\n\n\nName: Mario (Marvin) Theplumber\n\nemail: mario.thplumber@gmail.com\nphone: +72787226083\n\nNorth Holland, Netherlands\nhttps://github.c\n\nQ: what is in Cover letter?\nA:'

### Query the model manually

In [19]:
llm_client=InferenceClient(
                model="mistralai/Mistral-7B-Instruct-v0.2",
                token=HF_TOKEN
            )

NameError: name 'InferenceClient' is not defined

In [20]:
llm_question="what is this cover letter about?"

messages = [
    {
        "role": "user",
        "content": f"Based on this context:\n\n{context_text}\n\nAnswer: {llm_question}?"
    }
]

response = llm_client.chat_completion(
    messages=messages,
    max_tokens=500,
    temperature=0.7
)

answer = response.choices[0].message.content.strip()

NameError: name 'context_text' is not defined

In [73]:
response

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content=' This cover letter introduces the candidate, Mario (Marvin) Theplumber, and provides his contact information and links to his online profiles. It also mentions that he has created a resume using the "jsonresume-theme-caffeine-tweaked" and exported it as a PDF document named "resume.pdf". The last line provides his address in Rotterdam, Netherlands.\n\nTherefore, the cover letter is about introducing Mario Theplumber and providing his contact information, online profiles, and a mention of his resume creation.', reasoning=None, tool_call_id=None, tool_calls=None), logprobs=None)], created=1767728058183, id='8c8d87ff-028e-4547-bbd0-c7a93bb30d67', model='mistralai/Mistral-7B-Instruct-v0.2', system_fingerprint='', usage=ChatCompletionOutputUsage(completion_tokens=114, prompt_tokens=251, total_tokens=365), object='chat.completion')

In [74]:
answer

'This cover letter introduces the candidate, Mario (Marvin) Theplumber, and provides his contact information and links to his online profiles. It also mentions that he has created a resume using the "jsonresume-theme-caffeine-tweaked" and exported it as a PDF document named "resume.pdf". The last line provides his address in Rotterdam, Netherlands.\n\nTherefore, the cover letter is about introducing Mario Theplumber and providing his contact information, online profiles, and a mention of his resume creation.'

## List of questions

In [5]:
# import logging
from typing import Dict, List, Any

# logger = logging.getLogger(__name__)

def query_rag(
    question: str,
    text_processor: TextProcessor,
    hf_client: HuggingFaceClient,
    db_client: PgVectorClient,
    k: int = 5,
    # llm_question: str = None,
    max_new_tokens: int = 512,
    temperature: float = 0.7,
    similarity_threshold: float = 0.7,
) -> Dict[str, Any]:
    """
    Execute RAG query: embed → search → assemble context → generate answer.

    Args:
        question: User query
        text_processor: TextProcessor instance
        hf_client: HuggingFaceClient instance
        db_client: PgVectorDB instance
        k: Number of chunks to retrieve (default: 5)
        llm_question: Question to pass to the LLM (default: same as `question`)
        max_new_tokens: Maximum tokens for LLM response (default: 512)
        temperature: Temperature for LLM generation (default: 0.7)
        similarity_threshold: Minimum similarity score for retrieved chunks (default: 0.7)

    Returns:
        dict: {"answer": str, "sources": list, "num_chunks": int}

    Raises:
        ValueError: If embedding, search, or answer generation fails
    """
    try:
        logger.info(f"Processing RAG query for question: {question}")

        # Step 1: Generate embedding for the question
        logger.info("Generating query embedding...")
        query_embedding = hf_client.get_embeddings([question])[0]
        if not isinstance(query_embedding, list):
            query_embedding = query_embedding.tolist()
        logger.debug(f"Query embedding shape: {len(query_embedding)}")

        # Step 2: Search for relevant chunks
        logger.info(f"Searching for top-{k} chunks with similarity threshold: {similarity_threshold}...")
        chunks = db_client.search(query_embedding, k=k, similarity_threshold=similarity_threshold)
        logger.info(f"Found {len(chunks)} relevant chunks")

        # Step 3: Assemble context for the LLM
        logger.info("Assembling context for LLM...")
        context = text_processor.assemble_context(chunks, question=question)
        logger.debug(f"Assembled context length: {len(context)}")

        # Step 4: Generate answer using the LLM
        logger.info("Generating answer using LLM...")
        answer = hf_client.generate_answer(
            question=question,
            context=context,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
        )

        # Prepare the result
        sources = [chunk.get("source", "unknown") for chunk in chunks]
        result = {
            "answer": answer,
            "sources": sources,
            "num_chunks": len(chunks),
        }
        logger.info("RAG query completed successfully")
        return result

    except Exception as e:
        logger.error(f"Failed to process RAG query: {e}", exc_info=True)
        raise ValueError(f"Failed to process RAG query: {e}") from e


In [7]:
model_results={}

for question in questions:
    model_results[question] = query_rag(
        question=question,
        text_processor=text_processor,
        hf_client=hf_client,
        db_client=db_client,
        k=K,
        # llm_question="What is this cover letter about?",
        max_new_tokens=MAX_CONTEXT_TOKENS,
        temperature=TEMPERATURE,
        similarity_threshold=SIMILARITY_THRESHOLD,
    )
    logger.info(f"\n================================================================")
    logger.info(f"For question {question}, answer is {model_results[question]}")
    logger.info(f"================================================================\n")


INFO:__main__:Processing RAG query for question: What is mario's email?
INFO:__main__:Generating query embedding...
INFO:hf_client:Generated 1 embeddings
INFO:__main__:Searching for top-10 chunks with similarity threshold: 0.2...
INFO:__main__:Found 0 relevant chunks
INFO:__main__:Assembling context for LLM...
INFO:__main__:Generating answer using LLM...
INFO:hf_client:Generated answer (91 chars)
INFO:__main__:RAG query completed successfully
INFO:__main__:
INFO:__main__:For question What is mario's email?, answer is {'answer': "I'm sorry, but there is no information about Mario's email address in the context provided.", 'sources': [], 'num_chunks': 0}
INFO:__main__:================================================================

INFO:__main__:Processing RAG query for question: How long does shipping take?
INFO:__main__:Generating query embedding...
INFO:hf_client:Generated 1 embeddings
INFO:__main__:Searching for top-10 chunks with similarity threshold: 0.2...
INFO:__main__:Found 1 r

In [8]:
model_results

{"What is mario's email?": {'answer': "I'm sorry, but there is no information about Mario's email address in the context provided.",
  'sources': [],
  'num_chunks': 0},
 'How long does shipping take?': {'answer': 'The context provided does not mention anything about shipping times. Therefore, an answer cannot be determined based on the context alone.',
  'sources': ['policy.txt'],
  'num_chunks': 1},
 'Where there any projects with recommendation systems done by Mario?': {'answer': 'Yes, according to the context provided, Mario worked on a project involving the development of a multi-channel recommendation system for a large pharmaceutical client.',
  'sources': ['basic_info.md', 'basic_info.md'],
  'num_chunks': 2},
 'Does mario like data science?': {'answer': 'Yes, based on the context provided, Mario expresses his passion for data science and enjoys working as a data scientist, particularly in the areas of customer and financial analytics. He mentions his expertise in the full pipe

In [ ]:
### Manually created

# def query_rag(question, text_processor, hf_client, db_client, k=5):
#         """
#         Execute RAG query: embed → search → assemble context → generate answer.
        
#         Args:
#                 question: User query
#                 text_processor: TextProcessor instance
#                 hf_client: HuggingFaceClient instance
#                 db_client: PgVectorDB instance
#                 k: Number of chunks to retrieve
                
#         Returns:
#                 dict: {"answer": str, "sources": list, "num_chunks": int}
#         """

#         query_embedding = hf_client.get_embeddings([question])[0]
#         # if not isinstance(query_embedding, list):
#         #     query_embedding = query_embedding.tolist()
#         # query_embedding

#         chunks = db_client.search(query_embedding, k=K, similarity_threshold=SIMILARITY_THRESHOLD)
#         logger.info(f"Found: {len(chunks)} results")

#         logger.info(chunks)

#         # llm_question="what is this cover letter about?"
#         context = text_processor.assemble_context(chunks, question=llm_question)
#         logger.info(context)
#         # context=chunks[0]['content']

#         try:
#                 answer = hf_client.generate_answer(
#                         question=llm_question,
#                         context=context,
#                         max_new_tokens=MAX_CONTEXT_TOKENS,
#                         temperature=TEMPERATURE)
#         except:
#                 raise ValueError("Can't call a model")



In [ ]:
## 

# def query_rag(question, text_processor, hf_client, db_client, k=5):
#     """
#     Execute RAG query: embed → search → assemble context → generate answer.
    
#     Args:
#         question: User query
#         text_processor: TextProcessor instance
#         hf_client: HuggingFaceClient instance
#         db_client: PgVectorDB instance
#         k: Number of chunks to retrieve
        
#     Returns:
#         dict: {"answer": str, "sources": list, "num_chunks": int}
#     """

#     # 1. Embed question
#     query_embedding = hf_client.get_embeddings([question])[0]
#     if not isinstance(query_embedding, list):
#         query_embedding = query_embedding.tolist()
    
#     # 2. Search database with LOWER threshold
#     chunks = db_client.search(
#         query_embedding, 
#         k=k, 
#         similarity_threshold=0.3  # ← CHANGE: 0.7 → 0.3
#     )
    
#     if not chunks:
#         return {
#             "answer": "No relevant information found.",
#             "sources": [],
#             "num_chunks": 0
#         }
    
#     # 3. Assemble context
#     context = text_processor.assemble_context(chunks, question=question)
    
#     # 4. Generate answer
#     try:
#         answer = hf_client.generate_answer(question, context)
#         if not answer or len(answer) < 10:
#             answer = text_processor.create_fallback(chunks)
#     except:
#         answer = text_processor.create_fallback(chunks)
    
#     return {
#         "answer": answer,
#         "sources": chunks,
#         "num_chunks": len(chunks)
#     }



# result = query_rag(
#     question="what is refund policy?",
#     text_processor=text_processor,
#     hf_client=hf_client,
#     db_client=db_client,
#     k=5
# )

# print(result["answer"])



## Debug search

In [ ]:
# def test_vector_operations(db_client):
"""Test if pgvector operators work."""

with db_client.conn.cursor() as cur:
    # Test 1: Create dummy vectors
    cur.execute("""
        SELECT 
            ARRAY[1.0, 2.0, 3.0]::vector <=> ARRAY[1.0, 2.0, 3.0]::vector AS identical,
            ARRAY[1.0, 2.0, 3.0]::vector <=> ARRAY[4.0, 5.0, 6.0]::vector AS different
    """)
    identical, different = cur.fetchone()
    
    logger.info(f"Vector distance (identical): {identical:.4f} (should be 0.0)")
    logger.info(f"Vector distance (different): {different:.4f} (should be > 0)")
    
    # Test 2: Check if index exists
    cur.execute("""
        SELECT indexname, indexdef 
        FROM pg_indexes 
        WHERE tablename = 'documents' AND indexname = 'idx_embedding'
    """)
    index = cur.fetchone()
    if index:
        logger.info(f"Vector index exists: {index[0]}")
    else:
        logger.warning("Vector index not found")



INFO:__main__:Vector distance (identical): 0.0000 (should be 0.0)
INFO:__main__:Vector distance (different): 0.0254 (should be > 0)
INFO:__main__:Vector index exists: idx_embedding


In [ ]:
# def test_raw_search(db_client, hf_client, question="email"):
# """Test search with NO filtering."""

question= "This is a document with CV summary" #"email"

logger.info(f"Testing raw search for: '{question}'")
logger.info("=" * 80)

# Generate embedding
query_embedding = hf_client.get_embeddings([question])[0]
if not isinstance(query_embedding, list):
    query_embedding = query_embedding.tolist()

logger.info(f"Query embedding generated: {len(query_embedding)} dimensions")
logger.info(f"Sample values: {query_embedding[:5]}")

# Search without ANY threshold
with db_client.conn.cursor() as cur:
    cur.execute("""
        SELECT 
            id,
            source,
            LEFT(content, 100) as preview,
            1 - (embedding <=> %s::vector) AS similarity
        FROM documents
        ORDER BY embedding <=> %s::vector
        LIMIT 5
    """, (query_embedding, query_embedding))
    
    results = cur.fetchall()

if not results:
    logger.error("No results returned at all!")
    # return
else:
    logger.info(f"Found {len(results)} results:")
    for i, (doc_id, source, preview, sim) in enumerate(results, 1):
        logger.info(f"{i}. Similarity: {sim:.4f}")
        logger.info(f"   Source: {source} (ID: {doc_id})")
        logger.info(f"   Preview: {preview}...")




INFO:__main__:Testing raw search for: 'This is a document with CV summary'
INFO:__main__:================================================================================
INFO:hf_client:Generated 1 embeddings
INFO:__main__:Query embedding generated: 384 dimensions
INFO:__main__:Sample values: [-0.07514718919992447, 0.1503158062696457, -0.06211642175912857, 0.020352579653263092, 0.053853780031204224]
ERROR:__main__:No results returned at all!


In [ ]:
# Get actual text from database
with db_client.conn.cursor() as cur:
    cur.execute("SELECT content, embedding FROM documents WHERE id = 1")
    stored_content, stored_embedding = cur.fetchone()

logger.info(f"Stored content: {stored_content[:100]}...")
logger.info(f"Stored embedding type: {type(stored_embedding)}")
logger.info(f"Stored embedding sample: {stored_embedding[:5]}")

# Generate NEW embedding for same text
new_embedding = hf_client.get_embeddings([stored_content])[0]
if not isinstance(new_embedding, list):
    new_embedding = new_embedding.tolist()

logger.info(f"New embedding type: {type(new_embedding)}")
logger.info(f"New embedding sample: {new_embedding[:5]}")

# Compare them
with db_client.conn.cursor() as cur:
    cur.execute("""
        SELECT embedding <=> %s::vector AS distance
        FROM documents 
        WHERE id = 1
    """, (new_embedding,))
    
    distance = cur.fetchone()[0]
    logger.info(f"Distance between stored and regenerated: {distance:.6f}")
    logger.info(f"Similarity score: {1 - distance:.6f}")


INFO:__main__:Stored content: Our refund policy allows returns within 35 days of purchase. Full refunds are provided for unopened ...
INFO:__main__:Stored embedding type: <class 'str'>
INFO:__main__:Stored embedding sample: [-0.0
INFO:hf_client:Generated 1 embeddings
INFO:__main__:New embedding type: <class 'list'>
INFO:__main__:New embedding sample: [-0.04964632913470268, 0.007881461642682552, 0.06498601287603378, 0.02755117043852806, 0.05388107895851135]
INFO:__main__:Distance between stored and regenerated: 0.000000
INFO:__main__:Similarity score: 1.000000


In [ ]:
# Test 1: Check data exists
with db_client.conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM documents")
    print(f"Total documents: {cur.fetchone()[0]}")

    cur.execute("SELECT COUNT(*) FROM documents WHERE embedding IS NOT NULL")
    print(f"Documents with embeddings: {cur.fetchone()[0]}")

    cur.execute("SELECT vector_dims(embedding) FROM documents LIMIT 1")
    stored_dim = cur.fetchone()[0]
    print(f"Stored dim: {stored_dim}, Expected: {db_client.embedding_dim}")
    


Total documents: 3


In [ ]:
# Test 3: Raw vector search (no threshold)
query_embedding = hf_client.get_embeddings(["recommendation systems"])[0]
if hasattr(query_embedding, 'tolist'):
    query_embedding = query_embedding.tolist()

query_embedding_str = f"[{','.join(map(str, query_embedding))}]"

with db_client.conn.cursor() as cur:
    cur.execute("""
        SELECT content, 1 - (embedding <=> %s::vector) AS similarity
        FROM documents
        ORDER BY embedding <=> %s::vector
        LIMIT 5
    """, (query_embedding_str, query_embedding_str))
    
    for content, sim in cur.fetchall():
        print(f"Similarity: {sim:.4f} | {content[:80]}")


INFO:hf_client:Generated 1 embeddings


Total docs: 3
Sample content: Our refund policy allows returns within 35 days of purchase. Full refunds are provided for unopened 
Stored dim: 384, Expected: 384


In [ ]:
with db_client.conn.cursor() as cur:
    cur.execute("""
        SELECT column_name, data_type, udt_name
        FROM information_schema.columns
        WHERE table_name = 'documents' AND column_name = 'embedding'
    """)
    
    col_info = cur.fetchone()
    logger.info(f"Column name: {col_info[0]}")
    logger.info(f"Data type: {col_info[1]}")
    logger.info(f"UDT name: {col_info[2]}")